Αρχικοπιήση και Imports που ειναι απαραιτητα

In [1]:
import json
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 

import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")


In [2]:
print("Tensorflow-version:", tensorflow.__version__)

Tensorflow-version: 2.11.0-dev20220906


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  #Auto den xreiazetai an exoume polla vms

In [4]:
os.environ.pop('TF_CONFIG', None)

In [5]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

In [6]:

pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model

Ειαγωγή dataset (Την γράφουμε τον κώδικα της  εισαγωγής  σε ένα python αρχείο το οποίο θα χρησιμοποιηθεί από όλους τους workers)

In [14]:
%%writefile natural_setup.py

import json
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 

import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import random
import cv2
import math
import seaborn as sns

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,Convolution2D,BatchNormalization
from tensorflow.keras.layers import Flatten,MaxPooling2D,Dropout

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array

from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

import warnings
warnings.filterwarnings("ignore")

def natural_dataset():
  data=[]
  labels=[]
  random.seed(42)
  imagePaths = sorted(list(os.listdir("/content/drive/MyDrive/natural_images/")))
  random.shuffle(imagePaths)
  print(imagePaths)

  for img in imagePaths:
      path=sorted(list(os.listdir("/content/drive/My Drive/natural_images/"+img)))
      for i in path:
          image = cv2.imread("/content/drive/My Drive/natural_images/"+img+'/'+i)
          image = cv2.resize(image, (128,128))
          image = img_to_array(image)
          data.append(image)
          l = label = img
          labels.append(l)
  
  data = np.array(data, dtype="float32") / 255.0
  labels = np.array(labels)
  mlb = LabelBinarizer()
  labels = mlb.fit_transform(labels)
  print(labels[0])
  (xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.4,random_state=42)
  print(xtrain.shape, xtest.shape ,ytrain.shape)
  return xtrain , xtest ,ytrain ,ytest
  

	


def build_and_compile_cnn_model():
  model_d=DenseNet121(weights='imagenet',include_top=False, input_shape=(128, 128, 3)) 

  x=model_d.output

  x= GlobalAveragePooling2D()(x)
  x= BatchNormalization()(x)
  x= Dropout(0.5)(x)
  x= Dense(1024,activation='relu')(x) 
  x= Dense(512,activation='relu')(x) 
  x= BatchNormalization()(x)
  x= Dropout(0.5)(x)

  preds=Dense(8,activation='softmax')(x) #FC-layer
  model=Model(inputs=model_d.input,outputs=preds) #isos thelei anti gia model_d na balo base model

  for layer in model.layers[:-8]:
    layer.trainable=False
    
  for layer in model.layers[-8:]:
      layer.trainable=True
  model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()
  
  return model

Overwriting natural_setup.py


Μόνος ένας worker

In [17]:
import natural_setup
batch_size = 32
x_train, x_test , y_train ,y_test = natural_setup.natural_dataset()
single_worker_model = natural_setup.build_and_compile_cnn_model()
#single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)


['dog', 'flower', 'motorbike', 'person', 'cat', 'fruit', 'airplane', 'car']
[0 0 0 1 0 0 0 0]
(4139, 128, 128, 3) (2760, 128, 128, 3)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d_4 (ZeroPadding2  (None, 134, 134, 3)  0          ['input_3[0][0]']                
 D)                                                                                               
                                                                                                  
 conv1/conv (Conv2D)            (None, 64, 64, 64)   9408

In [18]:
anne = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
checkpoint = ModelCheckpoint('model.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)


datagen.fit(x_train)
# Fits-the-model
history = single_worker_model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
               steps_per_epoch=x_train.shape[0] //128,
               epochs=50,
               verbose=2,
               callbacks=[anne, checkpoint],
               validation_data=(x_train, y_train))

Epoch 1/50

Epoch 1: val_loss improved from inf to 0.08506, saving model to model.h5


32/32 - 422s - loss: 0.2910 - accuracy: 0.9195 - val_loss: 0.0851 - val_accuracy: 0.9732 - lr: 0.0010 - 422s/epoch - 13s/step
Epoch 2/50


KeyboardInterrupt: ignored

In [ ]:

data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 
train_generator = data_generator.flow(x_train, y_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size
 
r = single_worker_model.fit(train_generator, validation_data=(x_test, y_test),
              steps_per_epoch=steps_per_epoch, epochs=50)

Epoch 1/50
  25/1562 [..............................] - ETA: 8:00 - loss: 1.6131 - accuracy: 0.4412

KeyboardInterrupt: ignored

2 workers

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

gia to deutero

tf_config['task']['index']=1

to trito

tf_config['task']['index']=2

ktl

In [ ]:
json.dumps(tf_config)

'{"cluster": {"worker": ["localhost:12345", "localhost:23456"]}, "task": {"type": "worker", "index": 0}}'

Ekapideysh tou modelou

In [ ]:
strategy = tf.distribute.MultiWorkerMirroredStrategy()


In [ ]:
with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import cifar_setup

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = cifar_setup.mnist_dataset(global_batch_size)

with strategy.scope():
  # Model building/compiling need to be within `strategy.scope()`.
  multi_worker_model = cifar_setup.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

Writing main.py


In [ ]:
%%bash
ls *.py

main.py
mnist_setup.py


In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
# first kill any previous runs
%killbgscripts

All background processes were killed.


In [ ]:
%%bash --bg
python main.py &> job_0.log

In [ ]:
import time
time.sleep(10)

In [ ]:
%%bash
cat job_0.log

2022-09-04 15:03:39.917085: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917227: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-09-04 15:03:39.917243: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-09-04 15:03:42.023861: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

ta apo kato gia ton worker number 2

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

In [ ]:
bash
python main.py

meta kanoume experiment me sharding dld pos na xorisontai ta dta se kommatia

des https://www.tensorflow.org/tutorials/distribute/input#sharding

gia na kaneis xrisi checkpoints h na kaneis save to modelo  kai ta bari xrisimopoies https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/distribute/multi_worker_with_keras.ipynb#scrollTo=XVk4ftYx6JAO